<a href="https://colab.research.google.com/github/ozbej/food-analysis/blob/main/ingredient_NER_test_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingredient NER extraction evaluation

In [38]:
%%capture
!pip install transformers datasets seqeval

In [39]:
from google.colab import drive
from datasets import load_dataset, load_metric, DatasetDict
from transformers import AutoModelForTokenClassification, AutoTokenizer
import numpy as np
from ast import literal_eval
import nltk
from nltk.tokenize import word_tokenize
import torch

nltk.download('punkt')

drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/AIR/model.zip" -d "./"
!unzip "/content/drive/MyDrive/AIR/dataset_tagged_30k.zip" -d "./data"

Archive:  /content/drive/MyDrive/AIR/model.zip
  inflating: ./model/config.json     
  inflating: ./model/pytorch_model.bin  
  inflating: ./model/special_tokens_map.json  
  inflating: ./model/tokenizer_config.json  
  inflating: ./model/tokenizer.json  
  inflating: ./model/training_args.bin  
  inflating: ./model/vocab.txt       
Archive:  /content/drive/MyDrive/AIR/dataset_tagged_30k.zip
  inflating: ./data/dataset-train-tagged.csv  
  inflating: ./data/dataset-valid-tagged.csv  
  inflating: ./data/dataset-test-tagged.csv  


Define labels and label-index mapping

In [40]:
label_list = ["O", "B-ING", "I-ING"]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

Load dataset and turn `labels` column from string to array

In [41]:
dataset = load_dataset('csv', data_files={'test': 'data/dataset-test-tagged.csv'})

  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
def process_tagged_rows(row):
  row["labels"] = literal_eval(row["labels"])
  return row

dataset["test"] = dataset["test"].map(process_tagged_rows)

Load tokenizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForTokenClassification.from_pretrained(
    './model/',
    id2label=id2label,
    label2id=label2id
)

Since the defined tokenizer is subword tokenizer, we have to align labels based on `word_ids`

In [ ]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word: # Start of a new word
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None: # Special token
      new_labels.append(-100)
    else: # Same word as previous token
      label = labels[word_id]
      if label == 1: # If label is B-ING we change it to I-ING
        label = 2
      new_labels.append(label)

  return new_labels

In [ ]:
def tokenize_and_align_labels(row):
  try:
    tokens = literal_eval(row["recipe_tokenized"])
  except:
    tokens = row["recipe_tokenized"].replace(',"\\",', ",")
    tokens = literal_eval(tokens)
  tokenized_inputs = tokenizer(tokens, truncation=True, is_split_into_words=True, return_tensors="pt")
  labels = row["labels"]
  new_labels = []
  word_ids = tokenized_inputs.word_ids()
  row["labels"] = align_labels_with_tokens(labels, word_ids)
  return tokenized_inputs

Define a function for computing evaluation metrics

In [ ]:
metric_seqeval = load_metric("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric_seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

<ipython-input-10-fddd527f5ff9>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_seqeval = load_metric("seqeval")


Perform NER extraction on the test set and evaluate

In [ ]:
from transformers import pipeline

model_checkpoint = "./model/"

all_predictions = []
all_real = []

for count, row in enumerate(dataset["test"]):

  tokenized_inputs = tokenize_and_align_labels(row)

  outputs = model(**tokenized_inputs).logits
  predictions = torch.argmax(outputs, dim=2)

  labels_pred = [id2label[int(i)] for i in predictions[0]]
  labels_real = [id2label[i] if i != -100 else "O" for i in row["labels"]]

  if len(labels_pred) != len(labels_real): 
    print("Error, skipped line", count)
    continue

  all_predictions.append(labels_pred)
  all_real.append(labels_real)

metric_seqeval.compute(predictions=all_predictions, references=all_real)

{'ING': {'precision': 0.7799654576856649,
  'recall': 0.8302615250264282,
  'f1': 0.8043279827236903,
  'number': 21757},
 'overall_precision': 0.7799654576856649,
 'overall_recall': 0.8302615250264282,
 'overall_f1': 0.8043279827236903,
 'overall_accuracy': 0.9603418803418804}

# Infering

In [44]:
from transformers import pipeline

# Replace this with your own checkpoint
token_classifier = pipeline(
    "token-classification", model="./model/", aggregation_strategy="simple"
)

In [53]:
recipe = """After washing basil and tomatoes, blot them dry with clean paper towel.
Using a clean cutting board, cut tomatoes into quarters.
For marinade, place first six ingredients in a blender. Cover and process until well blended.
Place chicken breasts in a shallow dish, do not rinse raw. Cover with marinade. Cover dish. Refrigerate about 1 hour, turning occasionally.
Wash hands with soap and water after handling uncooked chicken.
Place chicken on an oiled grill rack over medium heat. Do not reuse marinades used on raw foods. Grill chicken 4-6 minutes per side. Cook until internal temperature reaches 165 °F as measured with a food thermometer."""

In [54]:
token_classifier(recipe)

[{'entity_group': 'ING',
  'score': 0.5222461,
  'word': 'basil',
  'start': 14,
  'end': 19},
 {'entity_group': 'ING',
  'score': 0.7998817,
  'word': 'tomatoes',
  'start': 24,
  'end': 32},
 {'entity_group': 'ING',
  'score': 0.8148517,
  'word': 'tomatoes',
  'start': 105,
  'end': 113},
 {'entity_group': 'ING',
  'score': 0.8433876,
  'word': 'chicken breasts',
  'start': 229,
  'end': 244},
 {'entity_group': 'ING',
  'score': 0.8524475,
  'word': 'chicken',
  'start': 418,
  'end': 425},
 {'entity_group': 'ING',
  'score': 0.8691172,
  'word': 'chicken',
  'start': 433,
  'end': 440},
 {'entity_group': 'ING',
  'score': 0.84621054,
  'word': 'chicken',
  'start': 530,
  'end': 537}]